In [1]:
import re
import requests
from bs4 import BeautifulSoup
import numpy as np
from base64 import b64decode, b64encode
from collections import defaultdict
from functools import partial, wraps
from hashlib import md5, sha1, sha256
from itertools import chain, combinations, combinations_with_replacement, \
    count, cycle, permutations, product
from math import ceil, cos, exp, floor, pi, sin, sqrt
from operator import itemgetter, mul
import pandas as pd

In [2]:
#1
with open("day_1.txt") as f:
    inp_1 = f.read().strip()
inp_1 = inp_1.replace(',', '').split()

d = {'N': {'L': 'W', 'R': 'E'},
     'S': {'L': 'E', 'R':'W'},
     'E': {'L': "N", 'R': "S"},
     'W': {'L': "S", 'R':'N'}}

def new_pos(pos, old_dir, inst):
    try:
        new_dir = d[old_dir][inst[0]]
        new_dist = int(inst[1:])
        new_tup = {'N': lambda x: (0, x), 'S': lambda x: (0, -x), 'E': lambda x: (x, 0), 'W': lambda x: (-x, 0)}
        pos = pos[0] + new_tup[new_dir](new_dist)[0], pos[1] + new_tup[new_dir](new_dist)[1]
        return pos, new_dir
    except Exception as e:
        print(e, pos, old_dir, inst)

start = (0,0)
direction = "N"
for inst in inp_1:
    start, direction = new_pos(start, direction, inst.strip())

print(start, abs(start[0]) + abs(start[1]))

(-123, -123) 246


In [3]:
#2
def solve_1b(inp, debug=False):
    import builtins
    print = builtins.print if debug else lambda *x: None
    start = (0,0)
    visited = set()
    direction = "N"
    visited.add(start)
    flag = False
    for inst in inp:
        if flag:
            break
        n_pos, n_dir = new_pos(start, direction, inst.strip())
        print('next_pos', n_pos, inst)
        if n_pos[0] == start[0]:
            if n_pos[1] > start[1]:
                for i in range(start[1] + 1, n_pos[1] + 1):
                    cur_pos = (n_pos[0], i)
                    print(cur_pos)
                    if cur_pos in visited:
                        print('break', cur_pos)
                        flag = True
                        break
                    visited.add(cur_pos)
            else:
                for i in range(start[1] - 1, n_pos[1] -1, -1):
                    cur_pos = (n_pos[0], i)
                    print(cur_pos)
                    if cur_pos in visited:
                        print('break', cur_pos)
                        flag = True
                        break
                    visited.add(cur_pos)
        else:
            if n_pos[0] > start[0]:
                for i in range(start[0] + 1, n_pos[0] + 1):
                    cur_pos = (i, n_pos[1])
                    print(cur_pos)
                    if cur_pos in visited:
                        print('break', cur_pos)
                        flag = True
                        break
                    visited.add(cur_pos)
            else:
                for i in range(start[0] - 1, n_pos[0] - 1, -1):
                    cur_pos = (i, n_pos[1])
                    print(cur_pos)
                    if cur_pos in visited:
                        print('break', cur_pos)
                        flag = True
                        break
                    visited.add(cur_pos)
        start, direction = n_pos, n_dir
    return cur_pos

res = solve_1b(inp_1)
print(res, abs(res[0]) + abs(res[1]))

(-109, -15) 124


In [4]:
#3
with open("day_2.txt") as f:
    inp_2 = f.read().strip()
inp_2 = inp_2.split()

def solve_2(inst_list, keypad, start):
    direc = {'U': (-1, 0), 'D': (1, 0), 'L': (0, -1), 'R': (0, 1)}
    for inst in inst_list:
        for letter in inst:
            x, y = direc[letter]
            x, y = max(start[0] + x, 0), max(start[1] + y, 0)
            try:
                if keypad[x][y] == 0:
                    raise IndexError
            except IndexError:
                pass
            else:
                start = x, y

            #print(start, keypad[start[0]][start[1]])
        print(keypad[start[0]][start[1]], end='')

solve_2(inp_2, [[1,2,3], [4,5,6], [7,8,9]], (1,1))

53255

In [5]:
#4
solve_2(inp_2, [[0,0,1,0,0], 
                    [0,2,3,4,0], 
                    [5,6,7,8,9], 
                    [0,'A','B','C',0],
                    [0,0,'D',0,0]], (2, 0))

7423A

In [6]:
#5
with open("day_3.txt") as f:
    inp3 = f.read().strip()
    
inp3a = [sorted(map(int, k.split())) for k in inp3.split('\n')]
len(list(filter(lambda x: x[0] + x[1] > x[2], inp3a)))

1050

In [7]:
#6
inp3b = inp3.split()
ninp = []

for i in range(len(inp3b)//3):
    if i % 3 == 0:
        ninp.append(sorted(list(map(int, [inp3b[i*3], inp3b[i*3 + 3], inp3b[i*3+6]]))))
    elif i % 3 == 1:
        ninp.append(sorted(list(map(int, [inp3b[(i-1)*3 + 1], inp3b[(i-1)*3 + 4], inp3b[(i-1)*3 + 7]]))))
    else:
        ninp.append(sorted(list(map(int, [inp3b[(i-2)*3 + 2], inp3b[(i-2)*3 + 5], inp3b[(i-2)*3 + 8]]))))

len(list(filter(lambda x: x[0] + x[1] > x[2], ninp)))

1921

In [8]:
#7
from collections import Counter
import re

with open("day_4.txt") as f:
    inp_4 = f.read().strip()
inp_4 = inp_4.split('\n')

def is_real(s, k):
    s = Counter(s.replace('-', ''))
    counts = sorted(s.items(), key=lambda x: (-x[1], x[0]))
    return ''.join(map(lambda x: x[0], counts[:5])) == re.findall(r'\[(.*)\]', k)[0]

total = 0
inp_filter = []
for ele in inp_4:
    s, k = ele.rsplit('-', 1)
    if is_real(s, k):
        num = int(re.findall(r'\d+', k)[0])
        inp_filter.append([s, num])
        total += num
        
print(total)

409147


In [9]:
#8
import string
def shift(s, k):
    s = s.split('-')
    k = k%26
    d = {}
    for c in string.ascii_lowercase:
        d[c] = (ord(c) - ord('a') + k)%26 + ord('a')
    table = str.maketrans(d)
    return ' '.join(i.translate(table) for i in s)

for s, k in inp_filter:
    b = shift(s, k)
    if "northpole" in b:
        print(k)
        break

991


In [10]:
#9
with open('day_5.txt') as f:
    salt = f.read().strip()
answer_2 = {}
answer_1 = []
start = 0

while len(answer_2) != 8:
    digest = md5((salt + str(start)).encode()).hexdigest()
    if digest.startswith('00000'):
        #print(digest)
        answer_1.append(digest[5])
        try:
            pos = int(digest[5])
        except ValueError:
            pos = None
        if pos is not None and pos < 8:
            if pos not in answer_2:
                answer_2[pos] = digest[6]
    start += 1

print(''.join(answer_1)[:8])

1a3099aa


In [11]:
#10
print(''.join(answer_2[k] for k in range(8)))

694190cd


In [12]:
#11
with open("day_6.txt") as f:
    inp_6 = f.read().strip().split('\n')
    
print(''.join(Counter(k).most_common()[0][0] for k in zip(*inp_6)))

gebzfnbt


In [13]:
#12
print(''.join(Counter(k).most_common()[-1][0] for k in zip(*inp_6)))

fykjtwyn


In [14]:
#13
with open("day_7.txt") as f:
    inp_7 = f.read().strip().split('\n')

def is_valid(s):
    braces = re.findall(r'\[(.*?)\]', s)
    non_braces = re.sub(r'(\[.*?\])', ' ', s).split()
    for brace in braces:
        try:
            a = re.findall(r'(.)(.)\2\1', brace)[0]
        except IndexError:
            a = ''
        if len(set(a)) == 2:
            return False
    for s in non_braces:
        try:
            a = re.findall(r'(.)(.)\2\1', s)[0]
        except IndexError:
            a = ''
        if len(set(a)) == 2:
            return True
    return False

print(sum(is_valid(k) for k in inp_7))

115


In [15]:
#14
def is_valid2(s):
    braces = re.findall(r'\[(.*?)\]', s)
    non_braces = re.sub(r'(\[.*?\])', ' ', s).split()
    to_check = []
    for brace in braces:
        for i, c  in enumerate(brace):
            a = re.findall(r'(.)(.)\1', brace[i:i+3])
            if a and len(set(a[0])) == 2:
                to_check.append(a[0][1]+a[0][0]+a[0][1])
    for check in to_check:
        for brace in non_braces:
            if check in brace:
                return True
    return False

print(sum(is_valid2(k) for k in inp_7))

231


In [16]:
#15
with open("day_8.txt") as f:
    inp_8 = f.read().strip().split('\n')
    
def modify(df, inst):
    def rect(df, inst):
        a,b = map(int, inst.split()[1].split('x'))
        df.loc[0:b-1, 0:a-1] = 1
    def rot(df, inst):
        inst = inst.split()
        b = int(inst[-1])
        a = int(inst[2].split('=')[1])
        if 'column' in inst:
            vals = list(df.loc[:,a].values)
            vals = vals[-b:] + vals[:-b]
            df.loc[:, a] = vals
        else:
            vals = list(df.loc[a,:].values)
            vals = vals[-b:] + vals[:-b]
            df.loc[a, :] = vals
        
    if 'rect' in inst:
        rect(df, inst)
    else:
        rot(df, inst)
        
df = pd.DataFrame(np.zeros((6, 50)))
for inst in inp_8:
    modify(df, inst)
df.sum().sum()

119.0

In [17]:
#16
df2 = df.replace(1.0, '*').replace(0, ' ')
for i in range(0, 50, 5):
    s = []
    for j in range(6):
        s.append(''.join(df2.loc[j,i:i+4]))
    print('\n'.join(s), '\n')
        

**** 
   * 
  *  
 *   
*    
****  

**** 
*    
***  
*    
*    
*     

*  * 
*  * 
**** 
*  * 
*  * 
*  *  

**** 
*    
***  
*    
*    
*     

 *** 
*    
*    
 **  
   * 
***   

**** 
*    
***  
*    
*    
*     

 **  
*  * 
*  * 
*  * 
*  * 
 **   

 **  
*  * 
*    
* ** 
*  * 
 ***  

***  
*  * 
*  * 
***  
*    
*     

 **  
*  * 
*  * 
*  * 
*  * 
 **   



In [18]:
#17
with open("day_9.txt") as f:
    inp_9 = f.read().strip()
    
def decompress(s, recur=False):
    i = 0
    tot = 0
    while i < len(s):
        if s[i] != '(':
            i += 1
            tot += 1
            continue
        reg = re.findall(r'^\((\d+)x(\d+)\)', s[i:])
        if not reg:
            i += 1
            tot += 1
            continue
        a, b = map(int, reg[0])
        t = a
        i += 3 + len(reg[0][0]) + len(reg[0][1])
        if recur:
            t = decompress(s[i: i+a], recur)
        tot += t*b
        i += a
    return tot

decompress(inp_9)

99145

In [19]:
#18
decompress(inp_9, True)

10943094568

In [20]:
#19


In [21]:
#20
